# TAREA NO. 3 - ESTADÍSTICA MULTIVARIADA: REGRESIÓN MULTIVARIADA

## 1. Contexto de la base de datos
Este conjutno de datos contiene seis meses de datos de consumo eléctrico de un hogar, recogidos entre enero y junio de 2007. Los datos incluyen información sobre potencia activa global, potencia reactiva global, tensión, intensidad global, submedición 1 (cocina), submedición 2 (lavadero) y submedición 3 (calentador de agua eléctrico y aire acondicionado). Con 260.640 mediciones en total, el objetivo es predecir o estimar el consumo eléctrico de los hogares.

| Nombre de la columna | Descripción |
| --- | --- |
| Fecha | La fecha de la observación. (Fecha) |
| Hora | La hora de la observación. (Hora) |
| Potencia_activa_global | La potencia activa total consumida por el hogar (kilovatios). (Numérico) |
| Potencia_reactiva_global | La potencia reactiva total consumida por el hogar (kilovatios). (Numérico) |
| Voltaje | El voltaje al cual se entrega la electricidad al hogar (voltios). (Numérico) |
| Intensidad_global | La intensidad de corriente promedio entregada al hogar (amperios). (Numérico) |
| Sub_medición_1 | La potencia activa consumida por la cocina (kilovatios). (Numérico) |
| Sub_medición_2 | La potencia activa consumida por la lavandería (kilovatios). (Numérico) |
| Sub_medición_3 | La potencia activa consumida por el calentador de agua eléctrico y el aire acondicionado (kilovatios). (Numérico) |

La variable respuesta para el modelo de regresión múltiple es la potencia aparente global en la red. A priori, esta variable no está incluida dentro del conjunto de datos, por lo que es necesario obtenerla a parir de la potencia reactiva y activa global que si están disponibles en el conjunto de datos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler

## 1. Análisis descriptivo de los datos

### 1.1. Preparación de los datos

In [ ]:
data = pd.read_csv('database.csv')
data.info()

In [ ]:
# Reemplazar valores '?' con NaN para identificar valores faltantes de manera uniforme
data.replace('?', np.nan, inplace=True)

# Convertir columnas numéricas de tipo object a float donde sea necesario
numeric_columns = [
    'Global_active_power',
    'Global_reactive_power',
    'Voltage',
    'Global_intensity',
    'Sub_metering_1',
    'Sub_metering_2',
]

for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Verificar los cambios y la cantidad de valores faltantes
missing_data_summary = data.isnull().sum()

missing_data_summary

In [ ]:
# Convertir las columnas Date y Time en un índice temporal para realizar la interpolación correctamente
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])
data.set_index('Datetime', inplace=True)

# Eliminar las columnas originales de fecha y hora para evitar redundancia
data.drop(columns=['Date', 'Time'], inplace=True)

# Aplicar interpolación lineal en todas las columnas numéricas
data_interpolated = data.interpolate(method='time')

# Verificar si quedan valores faltantes
remaining_missing = data_interpolated.isnull().sum()

remaining_missing

In [ ]:
data_interpolated['Global_apparent_power'] = np.sqrt(data_interpolated['Global_active_power']**2 + data_interpolated['Global_reactive_power']**2)
data_interpolated.drop(columns=['Global_active_power', 'Global_reactive_power'], inplace=True)
data_interpolated.head()

### 1.2. Análisis descriptivo

In [ ]:
# Análisis descriptivo de las columnas numéricas
descriptive_analysis = data_interpolated.describe()
descriptive_analysis


In [ ]:
import matplotlib.pyplot as plt

# Análisis exploratorio visual: Crear gráficas clave

# 1. Distribución de la Potencia Aparente Global
plt.figure(figsize=(10, 6))
plt.hist(data_interpolated['Global_apparent_power'], bins=50, edgecolor='k')
plt.title('Distribución de la Potencia Aparente Global')
plt.xlabel('Potencia Aparente Global (kW)')
plt.ylabel('Frecuencia')
plt.show()

# 2. Tendencia de la Tensión (Voltage) a lo largo del tiempo
plt.figure(figsize=(15, 6))
plt.plot(data_interpolated.index, data_interpolated['Voltage'], alpha=0.6)
plt.title('Tendencia de la Tensión a lo largo del Tiempo')
plt.xlabel('Tiempo')
plt.ylabel('Tensión (V)')
plt.show()

# 3. Relación entre Potencia Aparente Global y Corriente Global
plt.figure(figsize=(8, 6))
plt.scatter(
    data_interpolated['Global_apparent_power'],
    data_interpolated['Global_intensity'],
    alpha=0.6
)
plt.title('Relación entre Potencia Aparente Global y Corriente Global')
plt.xlabel('Potencia Aparente Global (kW)')
plt.ylabel('Corriente Global (A)')
plt.show()

# 4. Boxplot de Submeterings
plt.figure(figsize=(10, 6))
data_interpolated[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].boxplot()
plt.title('Distribución de Submeterings')
plt.ylabel('Consumo de Energía (Wh)')
plt.show()

In [ ]:
X = data_interpolated.drop(columns=['Global_apparent_power'])
y = data_interpolated['Global_apparent_power']

In [ ]:
X.shape, y.shape

### 1.3. Análisis de correlación de las variables

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = X.corr()

# Graficar un mapa de calor para visualizar las correlaciones
plt.figure()
plt.matshow(correlation_matrix, cmap='coolwarm', fignum=1)
plt.colorbar()
plt.xticks(range(correlation_matrix.shape[1]), correlation_matrix.columns, rotation=90)
plt.yticks(range(correlation_matrix.shape[0]), correlation_matrix.columns)
plt.title('Matriz de Correlación entre Variables', pad=20)
plt.show()

correlation_matrix

Correlaciones Bajas:
- Voltage tiene una correlación negativa moderada con Global_apparent_power y Global_intensity.

Correlación entre Submeterings:
- Las submediciones (Sub_metering_1, Sub_metering_2, y Sub_metering_3) presentan correlaciones bajas entre ellas, lo que sugiere que capturan consumos de energía en diferentes fuentes.

### 1.4. Reducción de la dimensionalidad

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Selección de las columnas numéricas para aplicar PCA
numeric_data = X.drop(columns=['index'])

# Escalar los datos para PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_data)

# Aplicar PCA para identificar el número óptimo de componentes
pca = PCA()
pca.fit(scaled_data)

# Calcular la proporción acumulada de varianza explicada
explained_variance_ratio = pca.explained_variance_ratio_.cumsum()

# Graficar la proporción acumulada de varianza explicada
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, marker='o')
plt.title('Proporción Acumulada de Varianza Explicada (PCA)')
plt.xlabel('Número de Componentes Principales')
plt.ylabel('Proporción de Varianza Explicada')
plt.grid()
plt.show()

# Elegir el número óptimo de componentes que expliquen al menos el 70% de la varianza
optimal_components = (explained_variance_ratio >= 0.7).argmax() + 1

# Aplicar PCA con el número óptimo de componentes
pca = PCA(n_components=optimal_components)
reduced_data = pca.fit_transform(scaled_data)

# Convertir los datos reducidos a un DataFrame para análisis posterior
pca_columns = [f'PC{i+1}' for i in range(optimal_components)]
reduced_df = pd.DataFrame(reduced_data, columns=pca_columns, index=data_interpolated.index)

reduced_df.head()


In [ ]:
# Crear un biplot para visualizar cómo las variables originales contribuyen a las componentes principales
# Para esto, se utiliza la proyección de las variables originales en el espacio de los componentes principales.

# Obtener los pesos (loadings) de las variables originales en las primeras dos componentes principales
loadings = pca.components_.T[:, :2]

# Crear el biplot
plt.figure(figsize=(12, 8))

# Gráfico de las muestras en el espacio de las primeras dos componentes principales
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], alpha=0.3, label='Datos (Proyección)')

# Añadir las contribuciones de las variables originales
for i, column in enumerate(numeric_data.columns):
    plt.arrow(0, 0, loadings[i, 0] * 5, loadings[i, 1] * 5, 
              color='r', alpha=0.5, head_width=0.1)
    plt.text(loadings[i, 0] * 5.5, loadings[i, 1] * 5.5, column, 
             color='r', ha='center', va='center')

plt.title('Biplot: Componentes Principales y Variables Originales')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.axhline(0, color='grey', linestyle='--', linewidth=0.5)
plt.axvline(0, color='grey', linestyle='--', linewidth=0.5)
plt.grid(alpha=0.3)
plt.legend()
plt.show()

El biplot muestra la proyección de los datos en las dos primeras componentes principales y las contribuciones de las variables originales a estas componentes:

- Los vectores rojos indican qué tanto y en qué dirección cada variable contribuye a las componentes principales.
- Las variables con vectores más largos tienen un mayor peso en las componentes.
- La proximidad entre vectores de variables sugiere correlación entre estas.

In [ ]:
axs, fig = plt.subplots(1, 3, figsize=(18, 6))

# Gráfico de la primera componente principal
sns.histplot(reduced_df['PC1'], bins=50, kde=True, color='b', ax=fig[0])
fig[0].set_title('Distribución de PC1')
fig[0].set_xlabel('PC1')
fig[0].set_ylabel('Frecuencia')

# Gráfico de la segunda componente principal
sns.histplot(reduced_df['PC2'], bins=50, kde=True, color='g', ax=fig[1])
fig[1].set_title('Distribución de PC2')
fig[1].set_xlabel('PC2')
fig[1].set_ylabel('Frecuencia')

# Gráfico de la tercera componente principal
sns.histplot(reduced_df['PC3'], bins=50, kde=True, color='r', ax=fig[2])
fig[2].set_title('Distribución de PC3')
fig[2].set_xlabel('PC3')
fig[2].set_ylabel('Frecuencia')

plt.show()

In [ ]:
axs, fig = plt.subplots(3, 1, figsize=(15, 10))

# Gráfico de la primera componente principal
sns.lineplot(data=reduced_df['PC1'], ax=fig[0])
fig[0].set_title('Tendencia de PC1')
fig[0].set_xlabel('Tiempo')
fig[0].set_ylabel('PC1')

# Gráfico de la segunda componente principal
sns.lineplot(data=reduced_df['PC2'], ax=fig[1])
fig[1].set_title('Tendencia de PC2')
fig[1].set_xlabel('Tiempo')
fig[1].set_ylabel('PC2')

# Gráfico de la tercera componente principal
sns.lineplot(data=reduced_df['PC3'], ax=fig[2])
fig[2].set_title('Tendencia de PC3')
fig[2].set_xlabel('Tiempo')
fig[2].set_ylabel('PC3')

plt.tight_layout()
plt.show()

## 2. Análisis de series de tiempo

### 2.1. Verificación de la estacionaridad

In [ ]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series, column_name):
    result = adfuller(series)
    p_value = result[1]
    print(f"Prueba de Dickey-Fuller para {column_name}:")
    print(f"  Estadístico ADF: {result[0]:.4f}")
    print(f"  Valor p: {p_value:.4f}")
    print(f"  Valores Críticos: {result[4]}")
    if p_value < 0.05:
        print(f"  La serie '{column_name}' es estacionaria (p < 0.05).\n")
    else:
        print(f"  La serie '{column_name}' no es estacionaria (p >= 0.05).\n")
        
for column in reduced_df.columns:
    check_stationarity(reduced_df[column], column)

### 2.2. División del conjunto de datos

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
# Usaremos un 80% para entrenamiento y 20% para prueba
train_size = int(len(reduced_df) * 0.8)

train_data = reduced_df.iloc[:train_size]
test_data = reduced_df.iloc[train_size:]

# Mostrar un resumen de las divisiones
train_summary = {
    "Train Start": train_data.index[0],
    "Train End": train_data.index[-1],
    "Test Start": test_data.index[0],
    "Test End": test_data.index[-1],
    "Train Size": len(train_data),
    "Test Size": len(test_data),
}

train_summary

Parece que hay un problema persistente con los cálculos en este entorno. Permítanme resumir los pasos típicos para dividir el conjunto de datos:

Dividir en entrenamiento y prueba:
- Utilizar el 80% de los datos para el entrenamiento y el 20% para las pruebas. Esto es habitual en los modelos de series temporales para 
evaluar el rendimiento predictivo.

Límite entrenamiento-prueba:
- La división respeta el orden temporal (sin barajar).

### 2.3. Generación del modelo ARIMA y ajuste de los parametros (d, p, q)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import warnings

# Ignorar advertencias para mantener la salida limpia
warnings.filterwarnings("ignore")

# Intentar graficar ACF y PACF para las tres componentes principales en un solo gráfico

lags = 40  # Número de lags para ACF y PACF
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

for i, component in enumerate(reduced_df.columns):
    # Graficar ACF
    plot_acf(reduced_df[component], lags=lags, ax=axes[i, 0])
    axes[i, 0].set_title(f'{component} - ACF')
    
    # Graficar PACF
    plot_pacf(reduced_df[component], lags=lags, ax=axes[i, 1])
    axes[i, 1].set_title(f'{component} - PACF')

plt.tight_layout()
plt.show()

In [ ]:
# from pmdarima import auto_arima

# # Determine optimal ARIMA parameters for each component
# for component in reduced_df.columns:
#     stepwise_fit = auto_arima(reduced_df[component], seasonal=False, trace=True)
#     print(f"Optimal parameters for {component}: {stepwise_fit.order}")